# Gallery Example: M/Erl/1 Linear Network

This example demonstrates a linear network with n queues in series:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Erlang with varying orders per queue
- **Servers**: 1 server per queue
- **Scheduling**: FCFS

The Erlang order increases at each queue, providing decreasing service variability.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_merl1_linear(n=2, Umax=0.9):
    """Create M/Erl/1-Linear queueing network"""
    model = Network('M/Erl/1-Linear')
    
    # Block 1: nodes
    nodes = []
    source = Source(model, 'mySource')
    nodes.append(source)
    
    for i in range(n):
        queue = Queue(model, f'Queue{i+1}', SchedStrategy.FCFS)
        nodes.append(queue)
    
    sink = Sink(model, 'mySink')
    nodes.append(sink)
    
    # Block 2: classes
    oclass = OpenClass(model, 'myClass')
    
    # Exponential arrivals
    source.set_arrival(oclass, Exp(1))
    
    # Create pyramid utilization pattern
    half = n // 2
    means = list(np.linspace(0.1, Umax, half))
    if n % 2 == 0:
        means = means + means[::-1]
    else:
        means = means + [Umax] + means[::-1]
    
    # Set Erlang service for each queue with increasing order
    for i in range(n):
        nodes[i+1].set_service(oclass, Erlang.fit_mean_and_order(means[i], i + 1))
    
    # Block 3: topology
    model.link(Network.serial_routing(*nodes))
    
    return model

# Create the model with 2 queues
model = gallery_merl1_linear(n=2)

## About M/Erl/1 Linear Networks

This model combines:
- **Poisson arrivals** (SCV = 1)
- **Erlang service** with increasing order (decreasing SCV)

With Erlang order k, SCV = 1/k:
- Queue 1: order 1, SCV = 1.0 (exponential)
- Queue 2: order 2, SCV = 0.5
- Queue n: order n, SCV = 1/n

Later queues have more regular service, potentially improving performance.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)